In [51]:
import pyspark
from pyspark.sql import SparkSession, Row

spark = SparkSession.builder.appName("MyFirstProgram").getOrCreate()
text_file = "C:\\Users\\Amit.Jain\\Downloads\\learn_Spark\\ml-100k\\u.data"
inputData_Text = spark.read.text(text_file)
parquet_file = "C:\\Users\\Amit.Jain\\Downloads\\part-00000-0c827b90-3d7d-4fea-8fae-6e77c3bcd63a-c000.snappy.parquet"
print(parquet_file)
Input_parquet = spark.read.parquet(parquet_file)
Input_parquet.createOrReplaceTempView("pharmacy")

pharmacy_payload = spark.sql("select * from pharmacy")
pharmacy_payload.take(5)
#myResultDataFrame.select("aggregateId","aggregateType", "eventType" , "payload" ).show()
payLoads = pharmacy_payload.select("payload" )
payLoads.select("payload" ).show()
jsondata = spark.read.json(payLoads.rdd.map(lambda row: row.payload))


C:\Users\Amit.Jain\Downloads\part-00000-0c827b90-3d7d-4fea-8fae-6e77c3bcd63a-c000.snappy.parquet
+--------------------+
|             payload|
+--------------------+
|{"Id":20,"Ncpdp":...|
+--------------------+



In [25]:
import pyspark
from pyspark.sql import SparkSession, Row

spark = SparkSession.builder.appName("MyFirstProgram").getOrCreate()
json_file = "C:\\Users\\Amit.Jain\\Downloads\\pharmacy.json"
print(json_file)
Input_json = spark.read.option("multiline","true").json(json_file)
Input_json.createOrReplaceTempView("Amit_json")
myResultDataFrame2 = spark.sql("select * from Amit_json")
myResultDataFrame2.show()


C:\Users\Amit.Jain\Downloads\pharmacy.json
+---------+-----------+------------+------------+------------------+----------+----------------------------+----------------------+-------------------------+-------------------------+---------------------------+---------------------+--------------------+--------------+--------+--------------------+---------------------+---------------+----+-------+---------------------+---------------+--------------+-----------------+----------+----+--------------------+--------------------------+--------------------+-------------------------+-------------------+-------------+--------------+--------------------+-------+---------+----------+-------------------+------------+-----+--------------+---------------------+-------+----------------------+-----------------+----------+--------------+----------+----+---------+----+-----------------------------+-------------------+-------+---------------+-----------------------+----------------+-------------+-------------+-

In [20]:
import pyspark
from pyspark.sql import SparkSession, Row

spark = SparkSession.builder.appName("workOnFakeFriends_csv").getOrCreate()
csv_file = "C:\\Users\\Amit.Jain\\Downloads\\learn_spark\\fakefriends.csv"
print(csv_file)
Input_csv = spark.read.csv(csv_file)
Input_csv.createOrReplaceTempView("Amit_csv")
myResultDataFrame = spark.sql("select * from Amit_csv")
myResultDataFrame.show()
myResultDataFrame.select("_c0")
myResultDataFrame.select("_c1").show()

C:\Users\Amit.Jain\Downloads\learn_spark\fakefriends.csv
+---+--------+---+---+
|_c0|     _c1|_c2|_c3|
+---+--------+---+---+
|  0|    Will| 33|385|
|  1|Jean-Luc| 26|  2|
|  2|    Hugh| 55|221|
|  3|  Deanna| 40|465|
|  4|   Quark| 68| 21|
|  5|  Weyoun| 59|318|
|  6|  Gowron| 37|220|
|  7|    Will| 54|307|
|  8|  Jadzia| 38|380|
|  9|    Hugh| 27|181|
| 10|     Odo| 53|191|
| 11|     Ben| 57|372|
| 12|   Keiko| 54|253|
| 13|Jean-Luc| 56|444|
| 14|    Hugh| 43| 49|
| 15|     Rom| 36| 49|
| 16|  Weyoun| 22|323|
| 17|     Odo| 35| 13|
| 18|Jean-Luc| 45|455|
| 19|  Geordi| 60|246|
+---+--------+---+---+
only showing top 20 rows

+--------+
|     _c1|
+--------+
|    Will|
|Jean-Luc|
|    Hugh|
|  Deanna|
|   Quark|
|  Weyoun|
|  Gowron|
|    Will|
|  Jadzia|
|    Hugh|
|     Odo|
|     Ben|
|   Keiko|
|Jean-Luc|
|    Hugh|
|     Rom|
|  Weyoun|
|     Odo|
|Jean-Luc|
|  Geordi|
+--------+
only showing top 20 rows



In [21]:
import pyspark
from pyspark.sql import SparkSession, Row

spark = SparkSession.builder.appName("workOnFakeFriends_csv").getOrCreate()
csv_file = "C:\\Users\\Amit.Jain\\Downloads\\learn_spark\\fakefriends.csv"
print(csv_file)

def mapper(line):
    fields=line.split(",")
    return Row(ID=int(fields[0]), Name = str(fields[1].encode("utf-8")), age = int(fields[2]), numFriends = int(fields[3]))

line = spark.sparkContext.textFile(csv_file)
person = line.map(mapper)

schemaPerson = spark.createDataFrame(person).cache()
schemaPerson.createOrReplaceTempView("person")
teenager= spark.sql("select * from person")

for teen in teenager:
    print(teen)

schemaPerson.groupBy("age").count().orderBy("age").show()

spark.stop()

C:\Users\Amit.Jain\Downloads\learn_spark\fakefriends.csv
Column<'ID'>
Column<'Name'>
Column<'age'>
Column<'numFriends'>
+---+-----+
|age|count|
+---+-----+
| 18|    8|
| 19|   11|
| 20|    5|
| 21|    8|
| 22|    7|
| 23|   10|
| 24|    5|
| 25|   11|
| 26|   17|
| 27|    8|
| 28|   10|
| 29|   12|
| 30|   11|
| 31|    8|
| 32|   11|
| 33|   12|
| 34|    6|
| 35|    8|
| 36|   10|
| 37|    9|
+---+-----+
only showing top 20 rows



In [80]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as func

spark = SparkSession.builder.appName("friendsByAge").getOrCreate()

csv_file = "C:\\Users\\Amit.Jain\\Downloads\\learn_spark\\fakefriends-header.csv"

lines = spark.read.option("header","true").option("inferSchema","true").csv(csv_file)
#lines.show()

friendsByAge = lines.select("age", "friends")
friendsByAge.groupBy("age").agg(func.round(func.avg("friends"), 2).alias("AverageFreinds")).sort("age").show()



+---+--------------+
|age|AverageFreinds|
+---+--------------+
| 18|        343.38|
| 19|        213.27|
| 20|         165.0|
| 21|        350.88|
| 22|        206.43|
| 23|         246.3|
| 24|         233.8|
| 25|        197.45|
| 26|        242.06|
| 27|        228.13|
| 28|         209.1|
| 29|        215.92|
| 30|        235.82|
| 31|        267.25|
| 32|        207.91|
| 33|        325.33|
| 34|         245.5|
| 35|        211.63|
| 36|         246.6|
| 37|        249.33|
+---+--------------+
only showing top 20 rows



In [111]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

spark = SparkSession.builder.appName("friendsByAge").getOrCreate()

csv_file = "C:\\Users\\Amit.Jain\\Downloads\\learn_spark\\1800.csv"

schema = StructType([ \
                     StructField("stationID", StringType(), True), \
                     StructField("date", IntegerType(), True), \
                     StructField("measure_type", StringType(), True), \
                     StructField("temperature", FloatType(), True)])

lines = spark.read.option("header","false").option("inferSchema","true").schema(schema).csv(csv_file)
#lines.printSchema()
#lines.show()

minTemps = lines.filter(lines.measure_type=="TMIN")
maxTemps = lines.filter(lines.measure_type=="TMAX")

minTemps.groupBy("stationID").min("temperature").alias("temperature").show()
minTemps.groupBy("stationID").agg(func.min("temperature").alias("min_temperature")).show()
maxTemps.groupBy("stationID").max("temperature").alias("temperature").show()

                                                  

+-----------+----------------+
|  stationID|min(temperature)|
+-----------+----------------+
|ITE00100554|          -148.0|
|EZE00100082|          -135.0|
+-----------+----------------+

+-----------+---------------+
|  stationID|min_temperature|
+-----------+---------------+
|ITE00100554|         -148.0|
|EZE00100082|         -135.0|
+-----------+---------------+

+-----------+----------------+
|  stationID|max(temperature)|
+-----------+----------------+
|ITE00100554|           323.0|
|EZE00100082|           323.0|
+-----------+----------------+



In [119]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

spark = SparkSession.builder.appName("totalSpentByCustomer").getOrCreate()

csv_file = "C:\\Users\\Amit.Jain\\Downloads\\learn_spark\\customer-orders.csv"

customerOrderSchema = StructType([ \
                                  StructField("cust_id", IntegerType(), True),
                                  StructField("item_id", IntegerType(), True),
                                  StructField("amount_spent", FloatType(), True)
                                  ])

lines = spark.read.option("header","false").option("inferSchema","true").schema(customerOrderSchema).csv(csv_file)
lines.printSchema()
lines.show()

totalByCustomer = lines.groupBy("cust_id").agg(func.round(func.sum("amount_spent"), 2) \
                                      .alias("total_spent"))

##lines.groupBy("cust_id").func.sum("amount_spent").show()
totalByCustomer.show()         

root
 |-- cust_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- amount_spent: float (nullable = true)

+-------+-------+------------+
|cust_id|item_id|amount_spent|
+-------+-------+------------+
|     44|   8602|       37.19|
|     35|   5368|       65.89|
|      2|   3391|       40.64|
|     47|   6694|       14.98|
|     29|    680|       13.08|
|     91|   8900|       24.59|
|     70|   3959|       68.68|
|     85|   1733|       28.53|
|     53|   9900|       83.55|
|     14|   1505|        4.32|
|     51|   3378|        19.8|
|     42|   6926|       57.77|
|      2|   4424|       55.77|
|     79|   9291|       33.17|
|     50|   3901|       23.57|
|     20|   6633|        6.49|
|     15|   6148|       65.53|
|     44|   8331|       99.19|
|      5|   3505|       64.18|
|     48|   5539|       32.42|
+-------+-------+------------+
only showing top 20 rows

+-------+-----------+
|cust_id|total_spent|
+-------+-----------+
|     31|    4765.05|
|     85|    